In [ ]:
# Packages
import tsplib95
import random
import matplotlib.pyplot as plt
import os
import gzip

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 4: Optimization with Genetic Algorithm

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez

Student: Natzaret Gálvez Rísquez

Implementing a genetic algorithm for the Traveling Salesman Problem (TSP) using the TSPLIB library 

In [ ]:
# File paths
tsp_file_path = 'C:/Users/Gari/Desktop/Assignments_NEC/A4/eil101.tsp'

# Load the dataset
dataset = tsplib95.load(tsp_file_path)

Genetic algorithm  for solving the Traveling Salesman Problem (TSP) using permutation representation and various selection, mutation, and crossover techniques

In [38]:
import numpy as np

def euclidean_distance(coord1, coord2):
    return np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def parse_dataset(dataset):
    node_coords = {}
    for node in dataset.node_coords:
        node_coords[node] = dataset.node_coords[node]
    return node_coords

def compute_distances(node_coords):
    num_cities = len(node_coords)
    distances = np.zeros((num_cities, num_cities))
    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distances[i - 1, j - 1] = euclidean_distance(node_coords[i], node_coords[j])
    return distances

def roulette_wheel_selection(population, distances):
    fitness_values = np.array([1 / total_distance(individual, distances) for individual in population])
    probabilities = fitness_values / np.sum(fitness_values)
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index]

def partially_mapped_crossover(parent1, parent2):
    n = len(parent1)
    start, end = sorted(np.random.choice(n, 2, replace=False))
    child1, child2 = [-1] * n, [-1] * n
    child1[start:end + 1] = parent1[start:end + 1]
    child2[start:end + 1] = parent2[start:end + 1]

    for i in range(n):
        if not np.any(np.equal(parent2[i], child1)):
            index = i
            while np.any(child1[index] != -1):
                index = np.where(parent1 == parent2[index])[0][0]
            child1[index] = parent2[i]
        if not np.any(np.equal(parent1[i], child2)):
            index = i
            while np.any(child2[index] != -1):
                index = np.where(parent2 == parent1[index])[0][0]
            child2[index] = parent1[i]

    return child1, child2

def inversion_mutation(solution):
    n = len(solution)
    start, end = sorted(np.random.choice(n, 2, replace=False))
    return np.concatenate((solution[:start], np.flip(solution[start:end + 1]), solution[end + 1:]))

In [ ]:
def genetic_algorithm(distances, population_size=100, max_generations=100):
    n = distances.shape[0]
    population = [list(np.random.permutation(range(n))) for _ in range(population_size)]  # Convert arrays to lists

    for generation in range(max_generations):
        # Selection
        selected_parents = [roulette_wheel_selection(population, distances) for _ in range(2)]

        # Crossover
        child1, child2 = partially_mapped_crossover(selected_parents[0], selected_parents[1])

        # Mutation
        mutation_rate = 0.01
        if np.random.rand() < mutation_rate:
            child1 = inversion_mutation(child1)
        if np.random.rand() < mutation_rate:
            child2 = inversion_mutation(child2)

        # Replace the worst individuals with the children
        distances_population = [total_distance(individual, distances) for individual in population]
        worst1_idx = np.argmax(distances_population)
        population.pop(worst1_idx)

        distances_population = [total_distance(individual, distances) for individual in population]
        worst2_idx = np.argmax(distances_population)
        population.pop(worst2_idx)

        population.extend([child1, child2])

    best_solution = min(population, key=lambda x: total_distance(x, distances))
    return best_solution

In [ ]:
if __name__ == "__main__":
    # Dataset
    # Parse the dataset
    node_coords = parse_dataset(dataset)
    # Compute distances
    distances = compute_distances(node_coords)
    #print("Best tour:", solution)
    #print("Total distance:", total_distance(solution, distances))